In [ ]:
%load_ext autoreload
%autoreload 2
# default_exp importers.matrix

In [ ]:
# export
from hashlib import sha256
from integrators.data.schema import *
from integrators.imports import *
from nbdev.showdoc import show_doc
import docker
import time

# Matrix

This class is responsible for the installation of [Matrix homeserver Synapser](https://github.com/matrix-org/synapse), including generating the configuration files and uploading them to the Pod, running a Matrix homeserver instance and registrating with it to get an access token.

In [ ]:
# export
class Matrix:
    
    def __init__(self, url, hostname, user, data_dir, client, my_uid, my_gid):
        self.url = url # Matrix url
        self.hostname = hostname # Matrix hostname
        self.user = user # Matrix username
        self.dir = data_dir # the directory for configuration files
        self.client = client # docker client
        self.my_uid = my_uid # non-root user uid
        self.my_gid = my_gid # non-root user gid
        
        assert self.url is not None
        assert self.hostname is not None
        assert self.user is not None
        assert self.dir is not None
        assert self.my_uid is not None
        assert self.my_gid is not None
        assert self.client is not None
        
    def config_matrix(self):        
        """Generate and update configuration files"""
        # generate files
        self.client.containers.run(
            "matrixdotorg/synapse:latest",
            auto_remove=True,
            command="generate",
            detach=True, 
            volumes={self.dir: {'bind': '/data', 'mode': 'rw'}},
            environment=[f"UID={self.my_uid}", f"GID={self.my_gid}", f"SYNAPSE_SERVER_NAME={self.hostname}", "SYNAPSE_REPORT_STATS=no"],
        )
        while not os.path.exists(f"{self.dir}/homeserver.yaml"):
            time.sleep(5)
        
        # configure files
        find_replace = {
            "#enable_registration: false": "enable_registration: true",
            "#app_service_config_files:": "app_service_config_files:",
            "#  - app_service_1.yaml": "  - /data/registration.yaml"
        }
        with open(f"{self.dir}/homeserver.yaml") as fin:
            with open(f"{self.dir}/homeserver-bak.yaml", 'w') as fout:
                for line in fin:
                    for key in find_replace:
                        if key in line:
                            line = line.replace(key, find_replace[key])
                    fout.write(line)  
                    
        os.remove(f"{self.dir}/homeserver.yaml")
        os.rename(f"{self.dir}/homeserver-bak.yaml", f"{self.dir}/homeserver.yaml")   
        
    def run_matrix(self, networkname):
        """Run Matrix homeserver"""
        self.client.containers.run(
            "matrixdotorg/synapse:latest", 
            detach=True, 
            network=networkname,
            restart_policy={'Name': 'on-failure'},
            ports={'8008': '8008'},
            volumes={self.dir: {'bind': '/data', 'mode': 'rw'}},
            environment=[f"UID={self.my_uid}", f"GID={self.my_gid}"],
            name=self.hostname
        )
        time.sleep(5)
        
    def upload_configs(self, pod_client):
        """Upload config files to the Pod, can be used for re-installation"""
        matrix_network = Network(externalId=self.hostname, name="Matrix")
        pod_client.create(matrix_network)
        
        os.chdir(self.dir)
        for filename in os.listdir(os.getcwd()):
            with open(filename,"rb") as f:
                data = f.read() # read entire file as bytes
                readable_hash = sha256(data).hexdigest()
                file = File(externalId=filename, sha256=readable_hash)
                pod_client.create(file)
                pod_client.upload_file(data)
                matrix_network.add_edge("resource", file) 
                f.close()     
        pod_client.create_edges(matrix_network.get_all_edges())      
        
    def register_user(self, password, pod_client):
        """Register an account with Matrix, store the username and token in the Pod"""
        body = {"username": self.user, "password": password, "auth": {"type":"m.login.dummy"}}
        result = requests.post(f"{self.url}/_matrix/client/r0/register", json=body)
        json = result.json()
        user_id = json["user_id"]
        token = json["access_token"]
        matrix_acc = Account(externalId=self.user, importJson=token, displayName=user_id, service="matrix")
        pod_client.create(matrix_acc)

In [ ]:
show_doc(Matrix.config_matrix)

<h4 id="Matrix.config_matrix" class="doc_header"><code>Matrix.config_matrix</code><a href="__main__.py#L21" class="source_link" style="float:right">[source]</a></h4>

> <code>Matrix.config_matrix</code>()

Generate and update configuration files

In [ ]:
show_doc(Matrix.run_matrix)

<h4 id="Matrix.run_matrix" class="doc_header"><code>Matrix.run_matrix</code><a href="__main__.py#L52" class="source_link" style="float:right">[source]</a></h4>

> <code>Matrix.run_matrix</code>(**`networkname`**)

Run Matrix homeserver

In [ ]:
show_doc(Matrix.upload_configs)

<h4 id="Matrix.upload_configs" class="doc_header"><code>Matrix.upload_configs</code><a href="__main__.py#L66" class="source_link" style="float:right">[source]</a></h4>

> <code>Matrix.upload_configs</code>(**`pod_client`**)

Upload config files to the Pod, can be used for re-installation

In [ ]:
show_doc(Matrix.register_user)

<h4 id="Matrix.register_user" class="doc_header"><code>Matrix.register_user</code><a href="__main__.py#L83" class="source_link" style="float:right">[source]</a></h4>

> <code>Matrix.register_user</code>(**`password`**, **`pod_client`**)

Register an account with Matrix, store the username and token in the Pod

## MatrixClient

This class is responsible for the interactions with Matrix's [Client-Server API](https://matrix.org/docs/spec/client_server/r0.6.1), such as fetching the information about WhatsApp contacts, chats and most recent messages.

In [ ]:
# export
class MatrixClient:
    
    def __init__(self, url, token):
        self.url = url # Matrix url
        self.token = token # token of Matrix account
        
        assert self.url is not None
        assert self.token is not None        
        
    def get_joined_rooms(self):
        """List all rooms the user joined"""
        try:
            result = requests.get(f"{self.url}/_matrix/client/r0/joined_rooms?access_token={self.token}")
            if result.status_code != 200:
                print(result, result.content)
                return False
            else:
                json = result.json()
                res = json["joined_rooms"] # only rooms with "joined" status
                return res
        except requests.exceptions.RequestException as e:
            print(e)
            return None    
    
    def get_joined_members(self, room_id):
        """List all joined members in a room"""
        try:
            result = requests.get(f"{self.url}/_matrix/client/r0/rooms/{room_id}/joined_members?access_token={self.token}")
            if result.status_code != 200:
                print(result, result.content)
                return False
            else:
                json = result.json()
                res = json["joined"] 
                return res
        except requests.exceptions.RequestException as e:
            print(e)
            return None          
        
    def send_messages(self, room_id, body):
        """Send a message to a joined room"""
        try:
            result = requests.post(f"{self.url}/_matrix/client/r0/rooms/{room_id}/send/m.room.message?access_token={self.token}", json=body)
            if result.status_code != 200:
                print(result, result.content)
                return None
            else:
                json = result.json()
                event_id = json["event_id"] 
                return event_id
        except requests.exceptions.RequestException as e:
            print(e)
            return None
            
    def get_event_context(self, room_id, event_id):
        """Fetch events after a certain event"""
        try:
            # limit to context event is set to 1
            result = requests.get(f"{self.url}/_matrix/client/r0/rooms/{room_id}/context/{event_id}?limit=1&access_token={self.token}")
            if result.status_code != 200:
                print(result, result.content)
                return None
            else:
                json = result.json()
                res = json["events_after"] # only show events after the specific event
                return res
        except requests.exceptions.RequestException as e:
            print(e)
            return None    
        
    def sync_events(self, next_batch):
        """Fetch all events in a room in a batch"""
        try:
            result = requests.get(f"{self.url}/_matrix/client/r0/sync?since={next_batch}&access_token={self.token}")
            if result.status_code != 200:
                print(result, result.content)
                return None
            else:
                json = result.json()
                return json
        except requests.exceptions.RequestException as e:
            print(e)
            return None
        
    def get_profile(self, user_id):
        """Fetch profile of a specific user"""
        try:
            result = requests.get(f"{self.url}/_matrix/client/r0/profile/{user_id}")
            if result.status_code != 200:
                print(result, result.content)
                return None
            else:
                json = result.json() 
                return json
        except requests.exceptions.RequestException as e:
            print(e)
            return None
        
    def get_room_state(self, room_id):
        """Fetch status of a room"""
        try:
            result = requests.get(f"{self.url}/_matrix/client/r0/rooms/{room_id}/state?access_token={self.token}")
            if result.status_code != 200:
                print(result, result.content)
                return None
            else:
                json = result.json()
                return json
        except requests.exceptions.RequestException as e:
            print(e)
            return None
        
    def download_file(self, uri):
        """Download media files"""
        try:
            result = requests.get(f"{self.url}/_matrix/media/r0/download/{HOSTNAME}/{uri}")
            if result.status_code != 200:
                print(result, result.content)
                return None
            else:
                file = result.content
                return file
        except requests.exceptions.RequestException as e:
            print(e)
            return None

In [ ]:
show_doc(MatrixClient.get_joined_rooms)

<h4 id="MatrixClient.get_joined_rooms" class="doc_header"><code>MatrixClient.get_joined_rooms</code><a href="__main__.py#L11" class="source_link" style="float:right">[source]</a></h4>

> <code>MatrixClient.get_joined_rooms</code>()

List all rooms the user joined

In [ ]:
show_doc(MatrixClient.get_joined_members)

<h4 id="MatrixClient.get_joined_members" class="doc_header"><code>MatrixClient.get_joined_members</code><a href="__main__.py#L26" class="source_link" style="float:right">[source]</a></h4>

> <code>MatrixClient.get_joined_members</code>(**`room_id`**)

List all joined members in a room

In [ ]:
show_doc(MatrixClient.send_messages)

<h4 id="MatrixClient.send_messages" class="doc_header"><code>MatrixClient.send_messages</code><a href="__main__.py#L41" class="source_link" style="float:right">[source]</a></h4>

> <code>MatrixClient.send_messages</code>(**`room_id`**, **`body`**)

Send a message to a joined room

In [ ]:
show_doc(MatrixClient.get_event_context)

<h4 id="MatrixClient.get_event_context" class="doc_header"><code>MatrixClient.get_event_context</code><a href="__main__.py#L56" class="source_link" style="float:right">[source]</a></h4>

> <code>MatrixClient.get_event_context</code>(**`room_id`**, **`event_id`**)

Fetch events after a certain event

In [ ]:
show_doc(MatrixClient.sync_events)

<h4 id="MatrixClient.sync_events" class="doc_header"><code>MatrixClient.sync_events</code><a href="__main__.py#L72" class="source_link" style="float:right">[source]</a></h4>

> <code>MatrixClient.sync_events</code>(**`next_batch`**)

Fetch all events in a room in a batch

In [ ]:
show_doc(MatrixClient.get_profile)

<h4 id="MatrixClient.get_profile" class="doc_header"><code>MatrixClient.get_profile</code><a href="__main__.py#L86" class="source_link" style="float:right">[source]</a></h4>

> <code>MatrixClient.get_profile</code>(**`user_id`**)

Fetch profile of a specific user

In [ ]:
show_doc(MatrixClient.get_room_state)

<h4 id="MatrixClient.get_room_state" class="doc_header"><code>MatrixClient.get_room_state</code><a href="__main__.py#L100" class="source_link" style="float:right">[source]</a></h4>

> <code>MatrixClient.get_room_state</code>(**`room_id`**)

Fetch status of a room

In [ ]:
show_doc(MatrixClient.download_file)

<h4 id="MatrixClient.download_file" class="doc_header"><code>MatrixClient.download_file</code><a href="__main__.py#L114" class="source_link" style="float:right">[source]</a></h4>

> <code>MatrixClient.download_file</code>(**`uri`**)

Download media files

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted basic.ipynb.
Converted importers.EmailImporter.ipynb.
Converted importers.Importer.ipynb.
Converted importers.MatrixClient.ipynb.
Converted importers.WhatsAppImporter.ipynb.
Converted importers.util.ipynb.
Converted index.ipynb.
Converted indexers.FaceRecognitionIndexer.ipynb.
Converted indexers.FacerecognitionIndexer.Photo.ipynb.
Converted indexers.GeoIndexer.ipynb.
Converted indexers.NoteListIndexer.NoteList.ipynb.
Converted indexers.NoteListIndexer.Parser.ipynb.
Converted indexers.NoteListIndexer.ipynb.
Converted indexers.NoteListIndexer.util.ipynb.
Converted indexers.indexer.ipynb.
Converted itembase.ipynb.
Converted pod.client.ipynb.
